# RainfallQC example

In [1]:
import rainfallqc
print(rainfallqc.__version__)

0.0.7


In [ ]:
import polars as pl
from rainfallqc import gauge_checks, comparison_checks, timeseries_checks, neighbourhood_checks

from load_example_data import load_local_gdsr_data, load_local_gdsr_metadata

In [3]:
GAUGE_ID = "DE_02483"
RAIN_COL = "rain_mm"

In [4]:
gdsr_data = load_local_gdsr_data(GAUGE_ID)
gdsr_metadata = load_local_gdsr_metadata(GAUGE_ID)

In [5]:
gdsr_data

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


In [6]:
gauge_checks.get_years_where_annual_mean_k_top_rows_are_zero(gdsr_data, "rain_mm", k=1800)

[2006, 2008, 2009]

In [7]:
gauge_checks.get_years_where_nth_percentile_is_zero(gdsr_data, "rain_mm", quantile=.8)

[2006, 2008, 2009]

In [8]:
gauge_checks.breakpoints_check(gdsr_data, "rain_mm")

0

In [9]:
gauge_checks.intermittency_check(gdsr_data, "rain_mm")

[2006, 2010]

In [10]:
gauge_checks.min_val_change(gdsr_data, "rain_mm", expected_min_val=0.1)

[2006, 2010]

In [11]:
gauge_checks.check_temporal_bias(gdsr_data, "rain_mm", time_granularity='weekday')

0

# Comparison checks

In [12]:
comparison_checks.check_annual_exceedance_etccdi_prcptot(gdsr_data, "rain_mm", gauge_lat=gdsr_metadata['latitude'], gauge_lon=gdsr_metadata['longitude'])

[0, 0, 0, 0, 0]

In [13]:
comparison_checks.check_annual_exceedance_etccdi_r99p(gdsr_data, "rain_mm", gauge_lat=gdsr_metadata['latitude'], gauge_lon=gdsr_metadata['longitude'])

[0, 4, 0, 0, 0]

In [14]:
wr_check = comparison_checks.check_exceedance_of_rainfall_world_record(gdsr_data, "rain_mm", time_res='hourly')

In [15]:
wr_check['world_record_check'].value_counts()

world_record_check,count
i32,u32
4,1051
1,23
0,42750


In [16]:
rx1day_check = comparison_checks.check_annual_exceedance_etccdi_rx1day(gdsr_data, "rain_mm", gauge_lat=gdsr_metadata['latitude'], gauge_lon=gdsr_metadata['longitude'])

In [17]:
rx1day_check['rx1day_check'].value_counts()

rx1day_check,count
i32,u32
1,6
4,1078
0,42740
